# Pancake Pridiction v2

In [33]:
import pandas as pd
import math
import statsmodels.api as sm

pd.set_option('display.max_rows', None)

In [34]:
import os
import sys

current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)
sys.path.append(project_dir) if project_dir not in sys.path else None

import pancake
psp = pancake.Prediction()

## Data Preparation

### Config

In [35]:
dataset_reconds = 300

### Fetching Data

In [36]:
round_columns = ["epoch",
                 "startTimestamp",
                 "lockTimestamp",
                 "closeTimestamp",
                 "lockPrice",
                 "closePrice",
                 "lockOracleId",
                 "closeOracleId",
                 "totalAmount",
                 "bullAmount",
                 "bearAmount",
                 "rewardBaseCalAmount",
                 "rewardAmount",
                 "oracleCalled"]

def get_history(_psp, current_epoch, back_in_time=100):
    start_epoch_history = current_epoch - 2 - back_in_time

    df_history_round = pd.DataFrame(columns=round_columns)
    for i in range(start_epoch_history, current_epoch - 1):
        df_round = _psp.get_round(i)
        df_history_round = df_history_round.append(df_round)

    df_history_round = df_history_round.sort_values('epoch', ascending=False)
    df_history_round = df_history_round.reset_index(drop=True)
    
    return df_history_round

#### Creating a dataset

In [37]:
current_epoch = psp.get_current_epoch()
df = get_history(psp, current_epoch, dataset_reconds)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled
0,43170.0,1.644253e+09,1.644253e+09,1.644253e+09,434.645416,434.470001,0.0,0.0,36.586349,21.076054,15.510297,15.510297,35.488758,True
1,43169.0,1.644252e+09,1.644253e+09,1.644253e+09,434.330048,434.645416,0.0,0.0,32.466759,19.955526,12.511232,19.955526,31.492756,True
2,43168.0,1.644252e+09,1.644252e+09,1.644253e+09,433.024841,434.330048,0.0,0.0,33.327854,19.188622,14.139233,19.188622,32.328018,True
3,43167.0,1.644252e+09,1.644252e+09,1.644252e+09,436.176025,433.024841,0.0,0.0,68.727737,10.106404,58.621334,58.621334,66.665901,True
4,43166.0,1.644252e+09,1.644252e+09,1.644252e+09,437.041931,436.176025,0.0,0.0,43.497219,37.730164,5.767055,5.767055,42.192303,True


### Adding result

In [38]:
def add_result(lockPrice, closePrice):
    if lockPrice == closePrice:
        # draw
        return 0
    elif lockPrice > closePrice:
        # bear
        return -1
    elif lockPrice < closePrice:
        # bull
        return 1
        
df["result"] = df.apply(lambda row : add_result(row["lockPrice"], row["closePrice"]), axis=1)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled,result
0,43170.0,1.644253e+09,1.644253e+09,1.644253e+09,434.645416,434.470001,0.0,0.0,36.586349,21.076054,15.510297,15.510297,35.488758,True,-1
1,43169.0,1.644252e+09,1.644253e+09,1.644253e+09,434.330048,434.645416,0.0,0.0,32.466759,19.955526,12.511232,19.955526,31.492756,True,1
2,43168.0,1.644252e+09,1.644252e+09,1.644253e+09,433.024841,434.330048,0.0,0.0,33.327854,19.188622,14.139233,19.188622,32.328018,True,1
3,43167.0,1.644252e+09,1.644252e+09,1.644252e+09,436.176025,433.024841,0.0,0.0,68.727737,10.106404,58.621334,58.621334,66.665901,True,-1
4,43166.0,1.644252e+09,1.644252e+09,1.644252e+09,437.041931,436.176025,0.0,0.0,43.497219,37.730164,5.767055,5.767055,42.192303,True,-1


### Maximum Consecutive Occurance

In [39]:
countDf = pd.DataFrame({'result': list(df['result'][(df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(df['result'].groupby((df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

bear_count = countDf[countDf.result == -1]['runs'].max()
bull_count = countDf[countDf.result == 1]['runs'].max()
draw_count = countDf[countDf.result == 0]['runs'].max()

print(f"Bear {bear_count} / Bull {bull_count} / Draw {draw_count}")

Bear 6 / Bull 7 / Draw nan


## Strategies

In [40]:
running_columns = ["epoch", "position", "amount", "reward", "recent_loss", "factor"]
df_running = pd.DataFrame(columns=running_columns)

In [41]:
def get_partial_history(df, current_epoch, back_in_time=6):
    df_history_round = df[(df.epoch < current_epoch) & (df.epoch >= current_epoch - back_in_time)]
    return df_history_round

    
def bet(df_running, epoch, position, amount, recent_loss, factor):
    data = [epoch, position, amount, 0, recent_loss, factor]
    temp = pd.DataFrame(data=[data], columns=running_columns)
    df_running = df_running.append(temp)
    return df_running

    
def get_round_stats(df, epoch):
    df_round = df[df.epoch == epoch]

    total_amount = (df_round["bullAmount"] + df_round["bearAmount"]).iloc[0]
    if total_amount > 0:
        bull_ratio = ((df_round["bullAmount"] / total_amount) * 100).iloc[0]
        bear_ratio = ((df_round["bearAmount"] / total_amount) * 100).iloc[0]
        bear_pay_ratio = (total_amount / df_round["bearAmount"]).iloc[0]
        bull_pay_ratio = (total_amount / df_round["bullAmount"]).iloc[0]
    else:
        bull_ratio = None
        bear_ratio = None
        bear_pay_ratio = None
        bull_pay_ratio = None

    lockPrice = df_round["lockPrice"].iloc[0]
    closePrice = df_round["closePrice"].iloc[0]

    if lockPrice == closePrice:
        # draw
        result = 0
    elif lockPrice > closePrice:
        # bear
        result = -1
    elif lockPrice < closePrice:
        # bull
        result = 1
    
    return {"total_amount": total_amount,
           "bull_ratio": bull_ratio, "bear_ratio": bear_ratio,
           "bear_pay_ratio": bear_pay_ratio, "bull_pay_ratio": bull_pay_ratio,
           "result": result,
           "lock_price": lockPrice,
           "close_price": closePrice}


def update_reward(df, df_running, epoch, status):
    bet_value = df_running[df_running.epoch == epoch]["amount"].iloc[0]
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    
    if status == 0:
        df_running.loc[df_running.epoch == epoch, "reward"] = -1 * bet_value
        pay_ratio = 0
    elif status == 1:
        epoch_stats = get_round_stats(df, epoch)
        if bet_position == "bull":
            pay_ratio = epoch_stats["bull_pay_ratio"]
        elif bet_position == "bear":
            pay_ratio = epoch_stats["bear_pay_ratio"]

        df_running.loc[df_running.epoch == epoch, "reward"] = (bet_value * pay_ratio) - bet_value
    
    print(f"[{epoch}] - Factor {pay_ratio} - Bet {bet_value} - Position {position}")    
    return df_running


def check_result(df, df_running, epoch):
    
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    epoch_stats = get_round_stats(df, epoch)
    
    if bet_position == "bull" and epoch_stats["result"] == 1:
        result = 1
    elif bet_position == "bear" and epoch_stats["result"] == -1:
        result = 1
    else:
        result = 0
    
    df_running = update_reward(df, df_running, epoch, result)
    return df_running, epoch_stats["result"]
    

def overview(df_running):
    total_spent = df_running.sum()["amount"]
    total_loss = abs(df_running[df_running["reward"] < 0].sum()["reward"])
    loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    estimated_win = df_running[df_running["reward"] > 0].sum()["reward"]
    win_times = df_running[df_running["reward"] > 0].count()["reward"]
    estimated_gain = df_running.sum()["reward"]
    max_spent = df_running.max()["amount"]

    last_win_epoch = df_running[df_running["reward"] > 0].max()["epoch"]
    if last_win_epoch is None or math.isnan(last_win_epoch):
        recent_loss = abs(df_running.sum()["reward"])
        recent_loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    else:
        recent_loss = abs(df_running[df_running["epoch"] > last_win_epoch].sum()["reward"])
        recent_loss_times = df_running[(df_running["epoch"] > last_win_epoch)
                                       & (df_running["reward"] < 0)].count()["reward"]

    return {"total_spent": total_spent,
            "max_spent": max_spent,
            "total_loss": total_loss,
            "loss_times": loss_times,
            "estimated_win": estimated_win,
            "win_times": win_times,
            "estimated_gain": estimated_gain,
            "recent_loss": recent_loss,
            "recent_loss_times": recent_loss_times}


### Strategies Default Settings

In [42]:
starting_epoch = df.min()["epoch"] + 20
maximum_epoch = df.max()["epoch"] - 1

# 0: Even / 1: Odd
strategy_remainder = 0
base_bet = 0.01
safe_bet = 0.2

current_epoch = starting_epoch
value = base_bet
print("Starting Epoch: ", starting_epoch)

Starting Epoch:  42890.0


### Strategy 1: Trend

In [43]:
def calculate_trend(df, current_epoch, back_in_time=6):
    df_history_round = get_partial_history(df, current_epoch, back_in_time)
    X = df_history_round.index
    X = X.astype(float)

    y = df_history_round['closePrice']
    y = y.astype(float)

    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    alpha = model.params['x1']
    return alpha

In [44]:
while current_epoch < maximum_epoch:
    if current_epoch % 2 == strategy_remainder:
        
        bet_status = overview(df_running)
        current_round_stats = get_round_stats(df, current_epoch)

        trend = calculate_trend(df, current_epoch, 3)
        print("Trend: ", trend)

        if trend < 0:
            # bull
            custom_factor = current_round_stats["bull_pay_ratio"] - safe_bet
            position = "bull"
        else:
            # bear
            custom_factor = current_round_stats["bear_pay_ratio"] - safe_bet
            position = "bear"

        value = (bet_status["recent_loss"] + base_bet) / (custom_factor - 1)
        if value < base_bet:
            value = base_bet

        df_running = bet(df_running, current_epoch, position, value, bet_status["recent_loss"], custom_factor)
        df_running, result = check_result(df, df_running, current_epoch)
        print(f"[{current_epoch}] - {position} - {value}")
    current_epoch += 1

Trend:  -0.3348846435547159
[42890.0] - Factor 1.8235752582550049 - Bet 0.016036556722894543 - Position bull
[42890.0] - bull - 0.016036556722894543
Trend:  -0.24465942382809658
[42892.0] - Factor 1.7725142240524292 - Bet 0.017466814936434188 - Position bull
[42892.0] - bull - 0.017466814936434188
Trend:  -0.40213012695306816
[42894.0] - Factor 1.9160943031311035 - Bet 0.013964641188004516 - Position bull
[42894.0] - bull - 0.013964641188004516
Trend:  -0.12571716308585223
[42896.0] - Factor 0 - Bet 0.020927157565035756 - Position bull
[42896.0] - bull - 0.020927157565035756
Trend:  -0.04142761230474434
[42898.0] - Factor 0 - Bet 0.04141934524306503 - Position bull
[42898.0] - bull - 0.04141934524306503
Trend:  0.22500610351568184
[42900.0] - Factor 0 - Bet 0.06915127919254763 - Position bear
[42900.0] - bear - 0.06915127919254763
Trend:  0.08296203613284092
[42902.0] - Factor 0 - Bet 0.2559679209531929 - Position bear
[42902.0] - bear - 0.2559679209531929
Trend:  -0.11225891113278408


### Strategy 2: Same-Before

### Strategy 3: Random

### Strategy 4: Exponential Moving Average (EMA)

In [45]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

## Overview

In [46]:
sum_reward = 0
df_running = df_running.reset_index(drop=True)

for index, row in df_running.iterrows():
        reward = row["reward"]
        sum_reward += reward
        
        df_running.loc[df_running.index == index, "acc_reward"] = sum_reward
        
df_running

,epoch,position,amount,reward,recent_loss,factor,acc_reward
0,42890.0,bull,0.016037,0.013207,0.000000,1.623575,0.013207
1,42892.0,bull,0.017467,0.013493,0.000000,1.572514,0.026701
2,42894.0,bull,0.013965,0.012793,0.000000,1.716094,0.039494
3,42896.0,bull,0.020927,-0.020927,0.000000,1.477848,0.018566
4,42898.0,bull,0.041419,-0.041419,0.020927,1.746684,-0.022853
5,42900.0,bear,0.069151,-0.069151,0.062347,2.046206,-0.092004
6,42902.0,bear,0.255968,-0.255968,0.131498,1.552795,-0.347972
7,42904.0,bull,0.463686,0.490203,0.387466,1.857187,0.142231
8,42906.0,bull,0.010698,-0.010698,0.000000,1.934784,0.131533
9,42908.0,bear,0.037403,0.028178,0.010698,1.553364,0.159712


In [47]:
summary = overview(df_running)
print(summary)

{'total_spent': 11.081631565320443, 'max_spent': 1.1954315216194606, 'total_loss': 3.375113029766308, 'loss_times': 72, 'estimated_win': 5.5376103165473145, 'win_times': 68, 'estimated_gain': 2.162497286781006, 'recent_loss': 0.04884640384079207, 'recent_loss_times': 3}


### Counting Consecutive Win/Loss

In [48]:
def add_result_binary(reward):
    if reward > 0:
        # bull
        return 1
    else:
        # bear
        return -1
        
temp_df = df_running.copy()

temp_df["result"] = temp_df.apply(lambda row : add_result_binary(row["reward"]), axis=1)

countDf = pd.DataFrame({'result': list(temp_df['result'][(temp_df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(temp_df['result'].groupby((temp_df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

loss_count = countDf[countDf.result == -1]['runs'].max()
win_count = countDf[countDf.result == 1]['runs'].max()

print(f"Loss {loss_count} / Win {win_count}")

Loss 5 / Win 4
